<a href="https://colab.research.google.com/github/thecodemancer/study-with-me/blob/main/gcp/getting_started_with_secrets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dealing with Secrets
<center>
<img src="https://www.apa.org/images/2020-09-exposing-secrets_tcm7-277839.jpg" />
</center>

## Setting up the development environment

In [ ]:
!pip install google.cloud

In [ ]:
!pip3 install --user google-cloud-secret-manager==2.10.0

In [ ]:
!gcloud auth list


No credentialed accounts.

To login, run:
  $ gcloud auth login `ACCOUNT`



In [ ]:
!gcloud config set account [YOUR EMAIL]

Updated property [core/account].


In [ ]:
!gcloud config list project

[core]
project (unset)

Your active configuration is: [default]


In [ ]:
!gcloud config set project formal-shell-295407

Are you sure you wish to set property [core/project] to formal-shell-295407?

Do you want to continue (Y/n)?  Y

Updated property [core/project].


In [ ]:
!gcloud config list project

[core]
project = formal-shell-295407

Your active configuration is: [default]


In [ ]:
!gcloud services enable secretmanager.googleapis.com # ERROR: (gcloud.services.enable) Your current active account [YOUR EMAIL] does not have any valid credentials. Please run: $ gcloud auth login

In [ ]:
!gcloud auth login

In [11]:
!gcloud services enable secretmanager.googleapis.com # OK

## Python Code

Let's start by importing the secretmanager library. If it doesn't work, restart and try again.

In [1]:
from google.cloud import secretmanager

In [34]:
"""
Utilities for interacting with Google Cloud services
"""
from typing import Any
import json
# import googleapiclient.discovery

# def get_project_number(project_name):
#   """Gets the project number from a project name."""
#  project_service = googleapiclient.discovery.build('cloudresourcemanager', 'v3')
#   request = project_service.projects().get(name='projects/{}'.format(project_name))
#   response = request.execute()
#   return response

def get_gcloud_secret_by_name(secret_name: str, gcloud_project: str, version: int = None) -> Any:
    """
    Get the value of a Google secret from Secret Manager. Version defaults to "latest" if not specified otherwise
    :param secret_name: Name of secret to retrieve
    :param gcloud_project: Name of google cloud project
    :param version: Version of secret, optional, defaults to latest version
    :return: secret value
    """
    if not version:
        version = "latest"

    client = secretmanager.SecretManagerServiceClient()

    secret_request_string = (
        f"projects/{gcloud_project}/secrets/{secret_name}/versions/{version}"
    )
    response = client.access_secret_version(name=secret_request_string)

    return response.payload.data.decode("utf-8")


Reading a secret that is a string

In [45]:
GOOGLE_CLOUD_PROJECT="formal-shell-295407"
SECRET_NAME="MYSECRET"
get_gcloud_secret_by_name(SECRET_NAME, GOOGLE_CLOUD_PROJECT)

'Hello secret!'

Reading a secret that is a key pair

In [37]:
SECRET_NAME="Compute-Engine-default-service-account-08f51050dbedd8bae5aa96d73ddb4780d5d571e2"
secrets = json.loads(get_gcloud_secret_by_name(SECRET_NAME, GOOGLE_CLOUD_PROJECT))

In [47]:
from google.cloud import storage
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_info(secrets)
client = storage.Client(project=GOOGLE_CLOUD_PROJECT, credentials=credentials)
INPUT_BUCKET="thecodemancer"
input_bucket = client.get_bucket(INPUT_BUCKET)
print(input_bucket)

<Bucket: thecodemancer>


In [39]:
dir(input_bucket)

['STORAGE_CLASSES',
 '_LOCATION_TYPES',
 '_MAX_OBJECTS_FOR_ITERATION',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_acl',
 '_changes',
 '_client',
 '_default_object_acl',
 '_encryption_headers',
 '_label_removals',
 '_location',
 '_patch_property',
 '_properties',
 '_query_params',
 '_require_client',
 '_set_properties',
 '_user_project',
 'acl',
 'add_lifecycle_abort_incomplete_multipart_upload_rule',
 'add_lifecycle_delete_rule',
 'add_lifecycle_set_storage_class_rule',
 'autoclass_enabled',
 'autoclass_toggle_time',
 'blob',
 'clear_lifecyle_rules',
 'client',
 'configure_website',
 'copy_blob',
 'cors',
 'create',
 'data_locations',
 'default_event_based_hold'

In [48]:
for blob in input_bucket.list_blobs():
  print(blob)

<Blob: thecodemancer, .bashrc, 1685361306391252>
<Blob: thecodemancer, .config/gcloud/.last_survey_prompt.yaml, 1685361306444219>
<Blob: thecodemancer, .config/gcloud/active_config, 1685361306451575>
<Blob: thecodemancer, .config/gcloud/configurations/config_default, 1685361306445216>
<Blob: thecodemancer, .config/gcloud/gce, 1685361306387695>
<Blob: thecodemancer, .config/gcloud/logs/2023.05.29/11.53.11.357715.log, 1685361306605289>
<Blob: thecodemancer, .docker/config.json, 1685361306687165>
<Blob: thecodemancer, .gsutil/credstore2, 1685361306728492>
<Blob: thecodemancer, .gsutil/credstore2.lock, 1685361306698490>
<Blob: thecodemancer, .gsutil/gcecredcache, 1685361306705492>
<Blob: thecodemancer, .gsutil/gcecredcache.lock, 1685361306738835>
<Blob: thecodemancer, Revelo/logo.jpeg, 1688600754824770>
<Blob: thecodemancer, Revelo/softwareengineering.png, 1673135048697310>
<Blob: thecodemancer, Untitled.ipynb, 1685361554830150>
<Blob: thecodemancer, input/, 1688606986444567>
<Blob: thecod

---
If you made it this far, follow [David Regalado](https://beacons.ai/davidregalado) for more code!